# Part II
## Multi-layer neural networks

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets
import random

In [3]:
from sklearn.datasets import make_blobs
X, t = make_blobs(n_samples=[400,800,400], centers=[[0,0],[1,2],[2,3]], 
                  n_features=2, random_state=2019)

indices = np.arange(X.shape[0])
random.seed(2020)
random.shuffle(indices)


X_train = X[indices[:800],:]
X_val = X[indices[800:1200],:]
X_test = X[indices[1200:],:]

t_train = t[indices[:800]]
t_val = t[indices[800:1200]]
t_test = t[indices[1200:]]


t2_train = t_train == 1
t2_train = t2_train.astype('int')
t2_val = (t_val == 1).astype('int')
t2_test = (t_test == 1).astype('int')

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s = scaler.transform(X_val)
P = len(X_train_s)

dim_hidden = 6 

dim_in =  len(X_train_s[0])
dim_out = len(set(t_train)) 

In [9]:
def logistic(x):
    return 1/(1+np.exp(-x))

In [133]:
def add_bias(X, b=1):
    # Put bias in position 0
    sh = X.shape
    if len(sh) == 1:
        #X is a vector
        return np.concatenate([np.array([1]), X])
    else:
        # X is a matrix
        m = sh[0]
        bias = np.ones((m,1)) # Makes a m*1 matrix of 1-s
        bias.fill(b)
        return np.concatenate([bias, X], axis  = 1)

##  Step 2: A Multi-layer neural network classifier

You want to train and test a classifier on (X, t). You could have put some parts of the code in the last step into a loop and run it through some iterations. But instead of copying code for every network we want to train, we will build a general Multi-layer neural network classfier as a class. This class will have some of the same structure as the classifiers we made for linear and logistic regression. The task consists mainly in copying in parts from what you did in step 1 into the template below. Remember to add the *self*- prefix where needed, and be careful in your use of variable names.

In [136]:
class MNNClassifier():
    """A multi-layer neural network with one hidden layer"""
    
    def __init__(self, eta=0.01, dim_hidden = 6):

        self.eta = eta
        self.dim_hidden = dim_hidden
        self.weights1 = None
        self.weights2 = None
        
        
    def fit(self, X_train, t_train, epochs = 100):

        dim_in =  len(X_train[0]) 
        dim_out = len(set(t_train))
        self.weights1 = np.random.normal(0, 0.5, [dim_in, dim_hidden+1]) #(L+ 1)×M weights between the input and the hidden layer 
        self.weights2 = np.random.normal(0, 0.5, [dim_hidden+1, dim_out]) #(M + 1)×N between the hidden layer and the output.
        self.weights1[:,0] = -1
        self.weights2[:,0] = -1
 
        
        for e in range(epochs):
            hidden_a, output_a = self.forward(X_train)

       
            one_hot = np.zeros((t_train.size, t_train.max()+1))
            one_hot[np.arange(t_train.size),t_train] = 1

      
            delta_o = (1 - output_a) * (one_hot - output_a) 
            delta_h = hidden_a * (1 - hidden_a) * (delta_o@self.weights2.T)
            self.weights2 += self.eta * hidden_a.T @ delta_o
            self.weights1 += self.eta * X_train.T @ delta_h 
              

                
    def forward(self, X):
    
        hidden_activations = logistic(X @ self.weights1)
        output_activations = logistic(hidden_activations @ self.weights2) 
        
        return hidden_activations, output_activations

    def predict(self, X_train):
        pred_o = self.forward(X_train)[1]
        one_hot_pred = np.zeros_like(pred_o)
        one_hot_pred[np.arange(len(pred_o)), pred_o.argmax(1)] = 1

        pred = np.arange(len(X_train))
    
        for i, e in enumerate(one_hot_pred):
            a = np.where(e == 1)
            pred[i] = a[0]
            
        return pred
        
    def accuracy(self, X_test, t_test):   
        return  np.mean(X_test==t_test)
        

mnn = MNNClassifier(dim_hidden = 6)
mnn.fit(X_train, t_train, epochs = 60)
predicted = mnn.predict(X_val)
print("Accuracy:", mnn.accuracy(predicted, t_val))


Accuracy: 0.75
